# Open Street Map - SF
***

Description: Using San Francisco, openStreet Map Data, to perform Audit and Analysis using SQL, Python.

## Map Area
***

San Francisco, CA

* https://mapzen.com/data/metro-extracts/metro/san-francisco_california/
* http://www.openstreetmap.org/relation/111968

Will be making that hometown soon, wanted to check it out.

## Process Dataset - On Sample
***

In [1]:
#Supporting Libraries
import os
import collections
import pprint
import xml.etree.cElementTree as ET
import re
import codecs
import csv
import cerberus
import copy
import schema

In [2]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-


#Provided by Project Instructions
import xml.etree.ElementTree as ET  # Use cElementTree or lxml if too slow

OSM_FILE = "dat/san-francisco_california.osm"  # Replace this with your osm file
SAMPLE_FILE = "dat/sf-sample.osm"

k = 10 # Parameter: take every k-th top level element

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


with open(SAMPLE_FILE, 'wb') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write('<osm>\n  ')

    # Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write('</osm>')

In [72]:
size_sf_osm=os.path.getsize('dat/san-francisco_california.osm')
size_sf_sample=os.path.getsize('dat/sf-sample.osm')

print "OSM Size: ",size_sf_osm
print "Sample SF: ",size_sf_sample

OSM Size:  966796961
Sample SF:  97786595


### Explore Sample
***

Counting different types of tags in the Sample File.

In [3]:
def count_tags(filename):
        tags=dict()
        for (event,elem) in ET.iterparse(filename):
            tag=elem.tag
            if tag:
                if tag not in tags.keys():
                    tags[tag] = 0
                tags[tag] = tags[tag] + 1
        return tags

In [4]:
sftags=count_tags(SAMPLE_FILE)
sftags

{'member': 3930,
 'nd': 538458,
 'node': 450946,
 'osm': 1,
 'relation': 507,
 'tag': 163525,
 'way': 52026}

Decided to look at the "k" to get all the different'key_type', such that we have a count of each of four tag categories in a dictionary:

* "lower", for tags that contain only lowercase letters and are valid,
* "lower_colon", for otherwise valid tags with a colon in their names,
* "problemchars", for tags with problematic characters, and
* "other", for other tags that do not fall into the other three categories.

In [5]:
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')


def key_type(element, keys):
    if element.tag == "tag":
        key=element.attrib['k']
        if lower.match(key):
            keys['lower']=keys['lower']+1
        elif lower_colon.match(key):
            keys["lower_colon"]=keys["lower_colon"]+1
        elif problemchars.match(key):
            keys["problemchars"]=keys["problemchars"]+1
        else:
            keys["other"]=keys["other"]+1
    return keys



def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)
    return keys

sf_keys= process_map(SAMPLE_FILE)
sf_keys

{'lower': 94378, 'lower_colon': 66603, 'other': 2544, 'problemchars': 0}

### Audit Sample
***

Looking at the Sample Data, I saw the following problems with the Data:

* Abbreviated Street Names: Street Names sometimes were abbreviated
* Phone Numbers: Phone numbers were inconsistent and did not follow the same schema.


#### Audit Street Names
***

I defined a mapping Dictionary where I provided full forms of various Abbreviations present in my Sample. And mapped it to the Street Types to make all street types consistent. 

In [6]:
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)


expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons"]

# UPDATE THIS VARIABLE
mapping = { "St": "Street",
            "St.": "Street",
            "Ave": "Avenue",
            "Blvd": "Boulevard",
            "Blvd.": "Boulevard",
            "Blvd,": "Boulevard",
            "Boulavard": "Boulevard",
            "Boulvard": "Boulevard",
            "Ct": "Court",
            "Dr": "Drive",
            "Dr.": "Drive",
            "E": "East",
            "Hwy": "Highway",
            "Ln": "Lane",
            "Ln.": "Lane",
            "Pl": "Place",
            "Plz": "Plaza",
            "Rd": "Road",
            "Rd.": "Road",
            "St": "Street",
            "St.": "Street",
            "st": "Street",
            "street": "Street",
            "square": "Square",
            "parkway": "Parkway"
            }


def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)


def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")


def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = collections.defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types


def update_name(name, mapping):
    r=street_type_re.search(name)
    if r:
        street_type=r.group()
        if street_type in mapping:
            name=re.sub(street_type_re,mapping[street_type],name)
    return name

Looking at all the different Street Types of our Sample

In [7]:
sf_street_types=audit(SAMPLE_FILE)
sf_street_types

defaultdict(set,
            {'15th': {'15th'},
             '200': {'Sutter Street, STE #200'},
             'A': {'Pier 50 A'},
             'Alameda': {'Alameda', 'The Alameda'},
             'Alley': {'Hodges Alley', "Kahn's Alley", 'Ross Alley'},
             'Ave': {'Blenheim Ave',
              'Magnolia Ave',
              'Palmetto Ave',
              'Phelan Ave',
              'Tehama Ave'},
             'Blvd': {'Airport Blvd',
              'Fremont Blvd',
              'Sawyer Camp Trail & Hillcrest Blvd'},
             'Blvd,': {'Nw Quad I-280 / Sr 35 Ic @ Jct Hayne Rd, Golf Course Dr, Skyline Blvd,'},
             'Bridgeway': {'Bridgeway'},
             'Broadway': {'Broadway'},
             'Center': {'Bon Air Center',
              'Embarcadero Center',
              'South Shore Center',
              'Westlake Center'},
             'Circle': {'Blossom Circle',
              'Citrus Circle',
              'Columbia Circle',
              'Croydon Circle',
         

In [8]:
for street_type, ways in sf_street_types.iteritems():
    for name in ways:
        better_name = update_name(name, mapping)
        print name, "=>", better_name

Bridgeway => Bridgeway
Buena Vista Avenue West => Buena Vista Avenue West
Industrial Parkway West => Industrial Parkway West
Vallejo => Vallejo
Fort Mason => Fort Mason
Ascot Rd => Ascot Road
townsend street => townsend Street
Prospect Way => Prospect Way
Ranleigh Way => Ranleigh Way
Koret Way => Koret Way
Lakemead Way => Lakemead Way
Wilmington Way => Wilmington Way
Brandy Rock Way => Brandy Rock Way
Christopher Way => Christopher Way
Berkeley Way => Berkeley Way
Black Fox Way => Black Fox Way
Abbott Way => Abbott Way
Madison Way => Madison Way
Windsor Way => Windsor Way
Dunsmuir Way => Dunsmuir Way
Embarcadero Way => Embarcadero Way
Boulevard Way => Boulevard Way
Sussex Way => Sussex Way
Kandle Way => Kandle Way
Mitchell Way => Mitchell Way
Granger Way => Granger Way
Eastlake Way => Eastlake Way
Bayridge Way => Bayridge Way
Cheshire Way => Cheshire Way
Bristol Way => Bristol Way
Martin Luther King Jr Way => Martin Luther King Jr Way
Chelsea Way => Chelsea Way
Mcnulty Way => Mcnulty W

#### Audit Phone Numbers
***

For phone, I first auditted for values that were not length 10 (US conventional Phone Number Length), and then performed mapping to convert phone numbers into 10 digits. 

In [58]:
def audit_phone(invalid_phone, phone):    
    if len(phone)!=10 or not phone[:3].isdigit():
        invalid_phone[phone]+=1

        
def is_phone(elem):
    return (elem.attrib['k'] == "phone")

def audit_phone_(osmfile):
    osm_file = open(osmfile, "r")
    invalid_phone = collections.defaultdict(int)
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_phone(tag):
                    audit_phone(invalid_phone,tag.attrib['v'])
    return invalid_phone

def update_phone(phone):
    new_phone=""
    for i in phone:
        if str(i) in ('1234567890'):
            new_phone+=str(i)
            new_phone=new_phone[-10:]
    return new_phone

In [40]:
sf_phone=audit_phone(SAMPLE_FILE)
sf_phone

defaultdict(int,
            {'(415) 252-9280': 1,
             '(415) 291-0333': 1,
             '(415) 338-0906': 1,
             '(415) 338-2049': 1,
             '(415) 338-2247': 1,
             '(415) 362-3128': 1,
             '(415) 398-1682': 1,
             '(415) 400-5614': 1,
             '(415) 421-0069': 2,
             '(415) 469-4520': 1,
             '(415) 487-9000': 1,
             '(415) 512-7424': 1,
             '(415) 563-0126': 1,
             '(415) 563-7284': 1,
             '(415) 585-5550': 1,
             '(415) 665-1355': 1,
             '(415) 730-7843': 1,
             '(415) 735 - 6843': 1,
             '(415) 742-5833': 1,
             '(415) 750-8446': 1,
             '(415) 759-2760': 1,
             '(415) 759-2811': 1,
             '(415) 771-0104': 1,
             '(415) 967-2622': 1,
             '(415)337-5555': 1,
             '(510) 261-2200': 1,
             '(510) 485-3054': 1,
             '(510) 536-6303': 1,
             '(510) 549-8820':

In [59]:
for phone in sf_phone.keys():
    new_phone=update_phone(phone)
    print phone, "=>", new_phone

(650) 556-9798 => 6505569798
+1 415-351-0175 => 4153510175
(415) 421-0069 => 4154210069
+1-510-658-7170 => 5106587170
(415) 487-9000 => 4154879000
+1 (415) 665-1008 => 4156651008
(650) 344-0284 => 6503440284
(650) 299-1243 => 6502991243
+1 415 292 9090 => 4152929090
+1 (415) 441-2200 => 4154412200
(650) 366-5394 => 6503665394
(650) 365-3962 => 6503653962
+1 (510) 749-0332 => 5107490332
(650) 369-4386 => 6503694386
(510) 261-2200 => 5102612200
+1 650 355 7093 => 6503557093
(415) 563-0126 => 4155630126
+1 510 452 1258 => 5104521258
+1 650-685-6620 => 6506856620
+1-415-781-6730 => 4157816730
415-285-7117 => 4152857117
(415) 759-2811 => 4157592811
415 7425122 => 4157425122
+1 415-885-6407 => 4158856407
415 986 2737 => 4159862737
+1 650 355 1555 => 6503551555
+1-510-547-5035 => 5105475035
(510) 536-6303 => 5105366303
+1 415 932 6715 => 4159326715
+1 415 665 6551 => 4156656551
+1 510 5499991 => 5105499991
+1 650 738 2100 => 6507382100
510-523-1777 => 5105231777
(415) 362-3128 => 4153623128
+

Its clear that the Data obtained from OpenMap is fairly clean, there is definitely room for improvement in the Phone Number and Street Names.

### Preparing for Database
***

Post auditting, it was time to convert the XML files into a CSV that can be easily imported into a SQL Database. 

In [79]:
NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema.Schema

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']


def shape_element(element, node_attr_fields = NODE_FIELDS, way_attr_fields = WAY_FIELDS,
                  problem_chars = PROBLEMCHARS, default_tag_type = 'regular'):
    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements

    # YOUR CODE HERE
    if element.tag == 'node':
        for n in NODE_FIELDS:
            node_attribs[n] = element.attrib[n]
        for c in element:
            n_tag = dict()
            if PROBLEMCHARS.search(c.attrib["k"]):
                continue
            else:
                if LOWER_COLON.search(c.attrib["k"]):
                    n_tag_type = c.attrib["k"].split(':',1)[0]
                    n_tag_key=c.attrib["k"].split(':',1)[1]
                    n_tag["key"]=n_tag_key
                    if n_tag_type:
                        n_tag["type"]=n_tag_type
                    else:
                        n_tag["type"]='regular'
                    n_tag["id"]=element.attrib["id"]
                    n_tag["value"]=c.attrib["v"]
                else:
                    n_tag["value"]=c.attrib["v"]
                    n_tag["key"]=c.attrib["k"]
                    n_tag["type"]="regular"
                    n_tag["id"]=element.attrib["id"]
                if n_tag:
                    tags.append(n_tag)
        return {'node': node_attribs, 'node_tags': tags}
    #Way
    elif element.tag == 'way':
        for w in WAY_FIELDS:
            way_attribs[w]=element.attrib[w]
        for c in element:
            w_tag=dict()
            wn=dict()
            if c.tag == 'tag':
                if PROBLEMCHARS.search(c.attrib["k"]):
                    continue
                else:
                    if LOWER_COLON.search(c.attrib["k"]):
                        w_tag_type=c.attrib["k"].split(':',1)[0]
                        w_tag_key=c.attrib["k"].split(':',1)[1]
                        w_tag["key"]=w_tag_key
                        if w_tag_type:
                            w_tag["type"]=w_tag_type
                        else:
                            w_tag["type"]='regular'
                        w_tag["id"]=element.attrib["id"]
                        w_tag["value"]=c.attrib["v"]
                    else:
                        w_tag["value"]=c.attrib["v"]
                        w_tag["key"]=c.attrib["k"]
                        w_tag["type"]="regular"
                        w_tag["id"]=element.attrib["id"]
                    if w_tag:
                        tags.append(w_tag)
            else:
                if c.tag=='nd':
                    wn["id"]=element.attrib["id"]
                    wn["node_id"]=c.attrib["ref"]
                    wn["position"]=len(way_nodes)
                if wn:
                    way_nodes.append(wn)
                else:
                    continue
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}


In [80]:
# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_strings = (
            "{0}: {1}".format(k, v if isinstance(v, str) else ", ".join(v))
            for k, v in errors.iteritems()
        )
        raise cerberus.ValidationError(
            message_string.format(field, "\n".join(error_strings))
        )


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])

                    
if __name__ == '__main__':
    # Note: Validation is ~ 10X slower. For the project consider using a small
    # sample of the map when validating.
    process_map(SAMPLE_FILE, validate = False)

##### File Size

In [82]:
nodes=os.path.getsize('nodes.csv')
nodes_tags=os.path.getsize('nodes_tags.csv')
ways=os.path.getsize('ways.csv')
ways_nodes=os.path.getsize('ways_nodes.csv')
ways_tags=os.path.getsize('ways_tags.csv')

print "Nodes File Size: ",nodes
print "Nodes Tags File Size: ",nodes_tags
print "Ways File Size: ",ways
print "Ways File Size: ",ways_nodes
print "Ways Tags File Size: ",ways_tags
print "OSM SF Fie Size: ",size_sf_osm
print "Sample SF File Size: ",size_sf_sample

Nodes File Size:  37923335
Nodes Tags File Size:  869155
Ways File Size:  3094728
Ways File Size:  12919302
Ways Tags File Size:  4818610
OSM SF Fie Size:  966796961
Sample SF File Size:  97786595


### Explore Database in SQL

In [121]:
import sqlite3
import pandas as pd

db = sqlite3.connect('sqlite/openstreetmap.db')
db.text_factory = lambda x: unicode(x, 'utf-8', 'ignore')

#DF
nodes_df = pd.read_csv(NODES_PATH)
node_tags_df =pd.read_csv(NODE_TAGS_PATH)
ways_df=pd.read_csv(WAYS_PATH)
way_nodes_df=pd.read_csv(WAY_NODES_PATH)
way_tags_df=pd.read_csv(WAY_TAGS_PATH)

#TO SQL
nodes_df.to_sql('nodes',db,index=True, if_exists='append',chunksize=1000)
node_tags_df.to_sql('node_tags',db,index=True, if_exists='append',chunksize=1000)
ways_df.to_sql('ways',db,index=True, if_exists='append',chunksize=1000)
way_nodes_df.to_sql('way_nodes',db,index=True, if_exists='append',chunksize=1000)
way_tags_df.to_sql('way_tags',db,index=True, if_exists='append',chunksize=1000)

db.commit()
db.close()

In [122]:
db=os.path.getsize('sqlite/openstreetmap.db')
print "SQL Database Size: ",db

SQL Database Size:  75653120


In [127]:
db = sqlite3.connect('sqlite/openstreetmap.db')
c = db.cursor()
c.execute("SELECT COUNT(*) FROM nodes;")
print "Number of Nodes: ",c.fetchall()

Number of Nodes:  [(450946,)]


In [128]:
c.execute("SELECT COUNT(*) FROM ways;")
print "Number of Ways: ",c.fetchall()

Number of Ways:  [(52026,)]


In [132]:
c.execute("SELECT COUNT(DISTINCT(j.uid)) FROM \
          (SELECT uid FROM nodes UNION SELECT uid FROM ways) j;")
print "Unique Users: ",c.fetchall()

Unique Users:  [(1410,)]


#### Major Restaurant Cuisine Type

In [145]:
c.execute("SELECT nt.value, COUNT(*) as n FROM node_tags as nt \
               JOIN (SELECT DISTINCT(id) FROM node_tags WHERE value = 'restaurant') i ON nt.id = i.id WHERE nt.key = 'cuisine'\
           GROUP BY nt.value\
           ORDER BY n DESC;")

print "Major Cuisine Type: ",c.fetchall()

Major Cuisine Type:  [(u'chinese', 16), (u'mexican', 16), (u'pizza', 16), (u'italian', 12), (u'thai', 10), (u'japanese', 9), (u'american', 8), (u'burger', 8), (u'sandwich', 7), (u'asian', 4), (u'regional', 4), (u'french', 3), (u'indian', 3), (u'korean', 3), (u'sushi', 3), (u'vietnamese', 3), (u'greek', 2), (u'peruvian', 2), (u'seafood', 2), (u'American', 1), (u'Fruit_Smoothies', 1), (u'Hispanic,_Spain,_Latin_Europe_cuisine', 1), (u'Japanese_Nabe', 1), (u'Juice', 1), (u'Vietnamnese', 1), (u'asain', 1), (u'barbecue', 1), (u'brazilian', 1), (u'californian', 1), (u'chinese;seafood', 1), (u'ethiopian', 1), (u'fondue', 1), (u'fried_food', 1), (u'ice_cream', 1), (u'japanese;korean', 1), (u'kebab', 1), (u'mediterranean', 1), (u'mexican_food', 1), (u'national', 1), (u'ramen', 1), (u'steak_house', 1)]
